In [1]:
%%capture

# update or install the necessary libraries

%pip install langgraph
%pip install --upgrade \
    langchain==0.1.14 \
    langchain-core==0.1.31 \
    langchain-openai==0.1.3
%pip install --upgrade python-dotenv

#### 기본 환경 설정

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

##### LLM 정의

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

##### 상태정의 

In [6]:
from typing import TypedDict

class QuestionState(TypedDict):
    messages: str

##### Agent / Tool Prompt 정의 

In [2]:
from langchain_core.tools import StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field

question_agent_prompt = """
your are a question agent app only can do an operation if you need other then call other app. Use Korean Language.
"""

mesage_context_prompt = """
This tool is a wrapper for answering messages.

**VERY IMPORTANT**  
Your input to this tool MUST message and string type. 

**Don't change result of json result by your self, Just return exact result.**
**Response format :** should be in JSON String.
"""

class MessageContext(BaseModel):
    messages: str = Field(..., description="messages")

def anwser_tool(messages) -> dict:
    result = llm.invoke(messages)
    print(result)

    return {
        "messages" : [
            result
        ],
    }

agent_tools = [
                StructuredTool.from_function(
                    anwser_tool, 
                    name="anwser", 
                    description=mesage_context_prompt, 
                    args_schema=MessageContext
                )
            ]

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


##### Agent 정의 

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor

def answer_question_agent_dict(state, agent, name):
    result = agent.invoke(state)

    # 기본적으로 출력되는 메시지
    output_state = {
        "messages": [HumanMessage(content=result["output"], name=name)]
    }
    return output_state


def answer_question_agent(llm, tools, system_message: str):
    """Create an app."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

##### Graph Build 

In [12]:
from langchain_core.messages import HumanMessage
from langgraph.constants import START, END
from langgraph.graph import StateGraph
import functools
  

workflow_graph_builder = StateGraph(QuestionState)
    
workflow_graph_builder.add_node(
    "answer_question", 
    functools.partial(
        answer_question_agent_dict, 
        agent=answer_question_agent(
            llm=llm,
            tools=agent_tools, 
            system_message=question_agent_prompt
        ), 
        name="answer_question"
    )
)

workflow_graph_builder.add_edge(START, "answer_question")
workflow_graph_builder.add_edge("answer_question", END)

workflow_app = workflow_graph_builder.compile()

result = workflow_app.invoke({
    "messages": [HumanMessage(content="안녕하세요?")]
})

print(result)

{'messages': [HumanMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}, name='answer_question')]}
